## PAN-STARRS Mbol Interpolation Using Color

Author: Aniket Sanghi

Institution: The University of Texas at Austin

This is general purpose notebook to interpolate bolometric magnitudes using PAN-STARRS photometry in g, r, i, z bands 
as the input. The interpolation is made utilizing Table B1. from ***Deacon, N.R., Kraus, A.L., Mann, A.W., et al. 2016, MNRAS, 455, 4212***

Follow the USER comments for instructions on utilizing this notebook. Users will commonly have to modify only cell 2. You will need to clone and download the entire repository so that the relevant text file (containing the grid) can be utilized for interpolation.

## User Inputs: ACTION REQUIRED

In [ ]:
# USER: Enter your system paths to the FITS file
fits_filepath = 'your_path/your_file.fits'

# USER: Enter the names of the columns storing the respective data in your FITS File
# If there are multiple columns for the same photometric band (arising from cross-matches etc.),
# specify names of additional columns in '' separated by commas
# The code will utilize the column that has photometry values and ignore ones that do not.
# NOTE: If you are specifying a photometric band column but do not have an error column, 
# add '' to the error column name. (You do not need to have multiple columns for all bands but you need to 
# have a corresponding error column for each specification even if you leave it as '')
# Example is provided below for multiple column use and no corresponding error column.

# PAN-STARRS Photometry
gmag_col = ['gmag', 'g_P1']
gmag_err_col = ['e_gmag', '']
rmag_col = ['rmag', 'r_P1']
rmag_err_col = ['e_rmag', 'r_P1_err']
imag_col = ['imag']
imag_err_col = ['e_imag']
zmag_col = ['zmag']
zmag_err_col = ['e_zmag']

# USER: If you have defined a subset for your data that is a boolean column and you wish to use those datapoints only. 
# Enter the name of the column and set flag to 1

# Subset Column
bool_subset_col = 'your_subset'
bool_subset_flag = 0

## Photometry Addition Function

In [ ]:
# USER: Do not modify contents

from astropy.io import fits
from astropy.table import Table
import numpy as np
from scipy.interpolate import griddata

def photometry(photometry_cols, photometry_dict):
    
    keys = list(photometry_dict.keys())
    
    for counter in range(0, len(photometry_cols), 2):
        
        photo_flag = 0
        
        for ctr in range(0, len(photometry_cols[counter])):
           
            if(photo_flag == 0):
                
                if(photometry_cols[counter][ctr] != '' and ~np.isnan(data[index][photometry_cols[counter][ctr]])):

                    photometry_dict[keys[counter]] = data[index][photometry_cols[counter][ctr]]
                    
                    if(photometry_cols[counter+1][ctr] != ''):
                        photometry_dict[keys[counter+1]] = data[index][photometry_cols[counter+1][ctr]]
                    
                    photo_flag = 1
            
            else:
                break
            
    return photometry_dict

## Access Data

In [ ]:
# Open FITS file
hdu_list = fits.open(fits_filepath, memmap=True)
data = hdu_list[1].data
hdu_list.close()

if(bool_subset_flag):
    data = data[data[bool_subset_col] == True]

## Interpolation

In [ ]:
mbol_interp = []

# Color calculations for model
g_model, r_model, i_model, z_model, y_model, mbol_model = np.loadtxt('text_files/PAN_STARRS_color_teff.txt', 
                                                                     unpack=True, usecols=(1,2,3,4,5,9))

g_r_model = np.subtract(g_model, r_model)
i_z_model = np.subtract(i_model, z_model)
    
for index in range(0, len(data)):

    # List of photometry column names
    photometry_cols = [gmag_col, gmag_err_col, rmag_col, rmag_err_col, imag_col, imag_err_col, zmag_col, zmag_err_col]

    # Initialized dictionary of photometry variables
    photometry_dict = {'gmag': np.nan, 'gmage':  np.nan, 'rmag':  np.nan, 'rmage':  np.nan, 'imag':  np.nan, 'image':  np.nan,
                       'zmag':  np.nan, 'zmage':  np.nan}

    # Color calculations for data
    photometry_dict = photometry(photometry_cols, photometry_dict)  
    g_r = np.subtract(photometry_dict['gmag'], photometry_dict['rmag'])
    i_z = np.subtract(photometry_dict['imag'], photometry_dict['zmag'])

    # Interpolate and append
    mbol = griddata((g_r_model, i_z_model), mbol_model, (g_r, i_z), method='linear')
    mbol_interp.append(mbol)

## Print Interpolated Data

In [ ]:
# Print interpolated data

mbol_interp = np.array(mbol_interp)
print(mbol_interp)